In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")


In [ ]:
train, info = tfds.load('cats_vs_dogs', split=('train[:80%]'), with_info=True, shuffle_files = True, as_supervised=True)
val = tfds.load('cats_vs_dogs', split=('train[80%:90%]'), shuffle_files = True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteAWW1EU/cats_vs_dogs-train.tfrecord


  0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
test, test_info = tfds.load('cats_vs_dogs', split=('train[90%:100%]'), with_info=True, shuffle_files = True, as_supervised=True)

In [ ]:
info

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    version=4.0.0,
    description='A large set of images of cats and dogs.There are 1738 corrupted images that are dropped.',
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=23262,
    splits={
        'train': 23262,
    },
    supervised_keys=('image', 'label'),
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
    title = {Asirra: A CAPTCHA that Exploits Interest-Aligned Manual Image Categorization},
    booktitle = {Proceedings of 14th ACM Conference on Computer and Communications Security (CCS)},
    ye

In [ ]:

ds = test.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  filename = example['image/filename']
  print(image.shape, label)
  print(filename)

['image', 'image/filename', 'label']
(500, 377, 3) tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(b'PetImages/Cat/8454.jpg', shape=(), dtype=string)


In [ ]:
#training pipeline
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def resize_img(image,label):
  return tf.image.resize(image, size=(224,224)), label

ds_train = train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.map(
    resize_img, num_parallel_calls=tf.data.AUTOTUNE)
#ds_train = ds_train.cache()
#ds_train = ds_train.shuffle(info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
#ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_val = val.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.map(resize_img, num_parallel_calls=tf.data.AUTOTUNE)
#ds_val = ds_val.cache()
#ds_val = ds_val.shuffle(info.splits['val'].num_examples)
ds_val = ds_val.batch(64)
#ds_val = ds_val.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_train

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
#evaluation pipeline
ds_test = test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(resize_img, num_parallel_calls=tf.data.AUTOTUNE)

ds_test = ds_test.batch(64)
#ds_test = ds_test.cache()
#ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False


In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation = 'relu')(x)
outputs = tf.keras.layers.Dense(2)(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dropout_1 (Dropout)         (None, 62720)             0         
                                                                 
 dense_2 (Dense)             (None, 32)                2007072   
                                                           

In [ ]:
history = model.fit(ds_train,
                    epochs=5,
                    validation_data=ds_val)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


146/146 [==============================] - 39s 249ms/step - loss: 0.2695 - accuracy: 0.9372 - val_loss: 0.0617 - val_accuracy: 0.9768
Epoch 2/5
146/146 [==============================] - 38s 256ms/step - loss: 0.1127 - accuracy: 0.9557 - val_loss: 0.0662 - val_accuracy: 0.9802
Epoch 3/5
146/146 [==============================] - 36s 245ms/step - loss: 0.0967 - accuracy: 0.9625 - val_loss: 0.0530 - val_accuracy: 0.9824
Epoch 4/5
146/146 [==============================] - 36s 243ms/step - loss: 0.0909 - accuracy: 0.9649 - val_loss: 0.0814 - val_accuracy: 0.9781
Epoch 5/5
146/146 [==============================] - 37s 255ms/step - loss: 0.0938 - accuracy: 0.9658 - val_loss: 0.0517 - val_accuracy: 0.9832


In [ ]:
loss, accuracy = model.evaluate(ds_test)


37/37 [==============================] - 4s 112ms/step - loss: 0.0427 - accuracy: 0.9858
